In [0]:
#Definição de bibliotecas
import pandas as pd
import pyspark.sql.functions as f
import sys
sys.path.append("../src")
from funcoes_pyspark import analisar_dataframe

###1 - Reading and Understanding Databases

Nosso objetivo: 1.Analisar os dados históricos de transações, ofertas e clientes

#### 1.1 - Offers Database

In [0]:
#Vamos ler a base de ofertas, reestruturar a coluna channels e organizar as colunas
offers = spark.createDataFrame(pd.read_json("../data/raw/offers.json"))\
    .withColumn("web",     f.when(f.array_contains("channels", "web"), 1).otherwise(0)) \
    .withColumn("email",   f.when(f.array_contains("channels", "email"), 1).otherwise(0)) \
    .withColumn("mobile",  f.when(f.array_contains("channels", "mobile"), 1).otherwise(0)) \
    .withColumn("social",  f.when(f.array_contains("channels", "social"), 1).otherwise(0))\
    .withColumnRenamed("id","offer_id")\
    .select("offer_id","offer_type","discount_value","min_value","duration","web","email","mobile","social").orderBy("offer_type")

In [0]:
#verificando volumetria e duplicatas de Id
analisar_dataframe(offers, "offer_id")

In [0]:
#Verificar saída da base de ofertas
offers.limit(5).display()

#####1.1.1 Data understanding

In [0]:
# offer_id: identificador único da oferta, usado para relacionar com eventos 
#           como recebimento, uso e expiração. É a chave para cruzar com outras bases.

# offer_type: tipo da oferta.
#   - "bogo" → Buy One Get One: compre um e ganhe outro, ou crédito equivalente.
#   - "discount" → desconto direto no valor da compra.
#   - "informational" → comunicação sem desconto, apenas informativa.

# discount_value: valor do benefício.
#   - Para "discount", é o desconto em moeda (ex.: 5 = R$ 5 de desconto). não é percentual!
#   - Para "bogo", discount_value = 10 → Ao comprar 1 item, o cliente recebe outro item ou crédito equivalente a R$ 10.
#   - Para "informational", normalmente é 0, pois não há desconto.

# min_value: valor mínimo de compra para ativar a oferta.
#            Ex.: min_value = 20 significa que a compra deve ser de pelo menos R$ 20
#            para o desconto ou brinde ser aplicado.

# duration: número de dias de validade da oferta a partir do envio ao cliente.

# web, email, mobile, social: canais de veiculação (1 = ativo, 0 = inativo).
# Permitem enviar a mesma oferta por múltiplos canais e medir qual funciona melhor.

# Essa tabela provavelmente é o catálogo de ofertas da empresa.
# Ela define todas as regras e parâmetros de cada promoção e serve de base 
# para cruzar com dados de eventos (quem recebeu, usou, etc.).
# Também permite análises como:
# - desempenho por tipo de oferta
# - eficácia por canal de comunicação
# - impacto de diferentes valores de desconto


#### 1.2 - Profiles Database

In [0]:
profile = spark.createDataFrame(pd.read_json("../data/raw/profile.json")).withColumnRenamed("id","profile_id").select("profile_id","registered_on","age","gender","credit_card_limit")

In [0]:
#verificando volumetria e duplicatas de Id
analisar_dataframe(profile, "profile_id")

In [0]:
profile.limit(5).display()

#####1.2.1 Data understanding

In [0]:
# profile_id: identificador único do cliente.
#             Usado para cruzar o perfil com outras tabelas (ex.: ofertas recebidas, transações, histórico de uso).

# registered_on: data de criação da conta (formato AAAAMMDD).
#                Representa o momento em que o cliente entrou na base.
#                Útil para calcular "tempo de relacionamento" e criar coortes.

# age: idade do cliente na data de criação da conta.
#      Valores extremamente altos (ex.: 118) indicam dado faltante ou inválido, será que é usado como código de "sem idade"? veremos mais na análise exploratória.
#      Pode ser ajustado via imputação ou categorização especial ("idade não informada").

# gender: gênero informado pelo cliente no cadastro.
#         Possíveis valores: "M" (masculino), "F" (feminino) ou null (não informado).
#         Null pode indicar falta de informação ou que o cliente optou por não declarar.
#         Nao posso inferir esse tipo de dados, mas pode ser usado para segmentação por gênero.

# credit_card_limit: limite de crédito do cartão registrado pelo cliente (float).
#                    Null indica ausência de cartão registrado ou dado indisponível.
#                    Pode ser usado como proxy de poder de compra ou perfil de risco.

# Observações gerais sobre a base:
# - São cerca de 17k clientes cadastrados.
# - Há campos com valores nulos ou inválidos, exigindo tratamento antes de análises/modelagem.
# - Não sei se essa idade na base cadastral é do momento do cadastro, ou se é atualizada. se for a data no momento do cadastro. A combinação "registered_on" + "age". Poderia nos dar a idade atual da pessoa, porém a base contém o campo credit_card_limit que deve ser atualizado, então vou tratar Age como a idade atual.
# - "credit_card_limit" pode ajudar em segmentações de clientes por capacidade de consumo.
# - A base serve como tabela de perfis, enriquecendo dados de ofertas e transações.


#### 1.3 - Transactions Database

In [0]:
from pyspark.sql.types import StructType, StringType, ArrayType
#Define o schema que representa o conteúdo da coluna "value"
#A coluna value contem o id da oferta. Os outros valores ("amount","reward") sao nulos em todo dataset
#Account_id deve ser análogo a profile_id entao vou renomear

transactions = spark.createDataFrame(pd.read_json("../data/raw/transactions.json")).withColumn("amount", f.col("value.amount")).withColumnRenamed("account_id","profile_id") \
    .withColumn("offer_id", f.col("value.`offer id`")).select("profile_id","offer_id","event","time_since_test_start")

In [0]:
analisar_dataframe(transactions, "profile_id")

In [0]:
transactions.limit(5).display()

#####1.3.1 Data understanding

In [0]:
# profile_id: id unico do cliente - originalmente era account_id
#             Serve pra ligar o evento a um cliente especifico.

# offer_id: id da oferta relacionada ao evento.
#           Aqui no exemplo já está como coluna, mas na base original vinha dentro da coluna "value" (json).
#           Usado pra cruzar com a tabela de ofertas e saber detalhes dela.

# event: tipo de evento que aconteceu.
#   - "transaction": cliente fez uma compra (pode ou não ter relação com oferta)
#   - "offer received": cliente recebeu uma oferta
#   - "offer viewed": cliente abriu/visualizou a oferta
#   - "offer completed": cliente cumpriu a condição da oferta (ex.: gastou o mínimo e ganhou o desconto)

# time_since_test_start: tempo desde o inicio do teste até o momento do evento, em dias.
#                        No exemplo tudo tá 0, então parece ser o registro inicial.

# value: na base completa, essa coluna guarda um json com informações adicionais:
#   - offer_id (quando é evento de oferta)
#   - reward/desconto (pra saber o valor do beneficio)
#   - amount (valor da transação quando é evento de compra)

# Essa tabela é um log de cerca de 300 mil eventos de clientes dentro do teste/campanha.
# A gte consegue seguir o "funil" de conversão: 
#   received → viewed → completed
#   e também ver transações que aconteceram com ou sem oferta vinculada.

In [0]:
transactions.groupBy("event","offer_id").count().orderBy("event").display()
#Essa coluna de evento junto com a "time_since_test_start" é capaz de definir a jornada do cliente após receber uma oferta!
#mas seguimos com algumas observações interessantes:

# - As "offer completed" sempre tem offer_id null → evento de conclusão sem oferta vinculada (provavelmente teremos que inferir pela janela de transação)
# - "transaction" com offer_id null → compra feita sem ligação com oferta ()
# - Em geral, número de "received" é bem maior que "viewed" e "completed"
# - Isso ajuda a achar ofertas que geram engajamento (view) mas não convertem em compra (completed)

# Possíveis usos dessa base:
# - Calcular taxas de conversão por oferta.
# - Avaliar quais ofertas têm melhor desempenho e quais flopam.
# - Cruzar com canais da oferta (web, email, mobile, social) pra ver qual canal traz mais resultado.

In [0]:
#Observando um cliente para tentar entender a jornada do cliente para diversas ofertas:
# Possíveis comportamentos por oferta (a partir do funil de eventos):
 # 1 - Na primeira oferta ele recebe > vê  > compra (da pra notar que o time_since_test_start de transaction == offer completed)
 # 2 - Ele faz uma transação sem vínculo a uma oferta
 # 3 - Ele recebe a oferta 9837 > vê > realiza 3 transações sem vínculo a oferta > deixa a oferta rolando
 # 4 - recebe oferta ddfd > vê > deixa rolando > recebe oferta e20d > nao le a oferta e20d > 1.25 dias depois ele transaciona (offer completed no mesmo dia, porem duplicada, pq se fossem duas comprar teríamos 2 transacoes no mesmo dia) alem disso nao sabemos se offer completed foi com relação  a ddfd ou  a 9837 (devemos olhar o tempo de vigência da oferta!!)
 # 5 - por fim ele finalmente visualiza a oferta e20d 

transactions.filter(f.col("profile_id")=="78afa995795e4d85b5d9ceeca43f5fef").orderBy("time_since_test_start").display()

#####1.3.2 - Unifying Databases

In [0]:
#Agora que entendemos individualmente o conceito das bases, vamos realizar o processso de enriquecimento de informações na base transacional para podermos avançar na construção da solução
#transactions, profile, offers
#Tomamos os valores distintos pois existem duplicidades na base,por exemplo temos 1 transacao para 2 ofertas completadas (mesma data hora), mesmo que seja possível finalizar 2 ofertas com uma transaçao, nao vamos considerar esses casos na solução.

transactions_enriquecida = transactions\
    .join(offers, on=["offer_id"], how="left") \
    .join(profile, on=["profile_id"], how="left").distinct()


In [0]:
analisar_dataframe(transactions_enriquecida, "profile_id")

In [0]:
#vamos analisar o cliente com mais transacoes na base
transactions.groupBy("profile_id").count().orderBy("count", ascending=False).limit(5).display()


In [0]:
#Vamos fazer uma análise parecida mas agora na base já enriquecida com informações de oferta e perfil

transactions_enriquecida.filter(f.col("profile_id")=="94de646f7b6041228ca7dec82adb97d2").orderBy("time_since_test_start").display()

In [0]:
#Como esperado, esse tipo de oferta não tem desconto nem valor mínimo, mas sim uma informação
transactions_enriquecida.filter(f.col("offer_type")=="informational").groupBy("discount_value","min_value").count().display()

In [0]:
#Os números em geral parecem fazer sentido:

# Ofertas Recebidas: Um total de 76,277 ofertas foram enviadas aos clientes.

#Ofertas Visualizadas: Dessas, 57,725 ofertas foram visualizadas pelos clientes, representando uma taxa de visualização de 75.6%. Isso #mostra que uma grande parte das ofertas chamou a atenção do público.

#Ofertas Completadas: Dentre as ofertas visualizadas, 30,617 ofertas resultaram em uma ação completada pelos clientes, gerando uma taxa #de conversão de 40.1%. Isso indica que muitos dos clientes que visualizaram as ofertas decidiram aproveitar alguma delas.

#Transações Totais: Considerando todas as transações realizadas, um total de 138,953 transações foi registrado. Dentre essas, 30,617 #transações foram associadas a ofertas, o que representa 22.1% das transações totais.
transactions_enriquecida.groupBy("event").count().display()


In [0]:
transactions_enriquecida.groupBy("event","offer_type").count().display()


###2 - Plano, Público e Target

Nosso objetivo: 2. Desenvolver uma técnica/modelo que auxilie na decisão de qual oferta enviar para cada cliente

####2.1 - Plano

Agora que temos uma base transacional enriquecida e já compreendemos as informações precisamos definir o que faremos para resolver o problema proposto "Qual oferta enviar para cada cliente?". <br><br>
Proponho então um **modelo supervisionado de target binário** que preveja a probabilidade de conversão.<br>
Cada linha representará um cliente recebendo uma oferta específica, ou seja o mesmo cliente poderá<br>
aparecer diversas vezes para cada tipo de oferta.<br>

Na hora de escorar vamos simular todas as ofertas possíveis e ver qual tem maior probabilidade de converter o cliente.<br>

Transações sem vínculo de oferta serão descartadas.


###2 - Público

In [0]:
transactions.limit(3).display()

In [0]:
#Vamos definir nosso público, composto na granularidade de clientes e oferta
#Clientes que receberam pelo menos uma opferta
publico = transactions.filter(f.col("offer_id").isNotNull()).withColumn("ID", f.concat_ws("_", "profile_id", "offer_id")).select("ID","profile_id","offer_id").distinct()

In [0]:
publico.limit(5).display()

In [0]:
analisar_dataframe(publico, "ID")

In [0]:
#Salvar o público para uso futuro
publico.toPandas().to_csv("../data/processed/publico.csv", sep=",",header=True,index=False)